In [1]:
# imports
import pandas as pd
import pickle
import openai
from typing import List, Dict, Tuple
from tempfile import TemporaryDirectory
import numpy as np
from gensim.models import KeyedVectors
from typing import List
import json


In [2]:
users = pd.read_csv("data_embeddings/users.csv") #document with user interactions
users.head()

,User,Interactions
0,U10045,"[-0.021019432693719864, -0.020645752549171448,..."
1,U11306,"[-0.022429920732975006, -0.028805548325181007,..."
2,U13000,"[-0.018485717475414276, -0.00828414037823677, ..."
3,U13740,"[-0.02095220796763897, -0.02534387819468975, 0..."
4,U1376,"[-0.042902905493974686, -0.01893831230700016, ..."


In [3]:
news = pd.read_csv("data_embeddings/news_sorted.csv") #document with user interactions
news.head()

,ID,Category,SubCategory,Content
0,N55528,lifestyle,lifestyleroyals,"[0.0059931655414402485, -0.007832585833966732,..."
1,N19639,health,weightloss,"[-0.004847756586968899, -0.00798266101628542, ..."
2,N61837,news,newsworld,"[-0.027519572526216507, -0.01374641340225935, ..."
3,N53526,health,voices,"[-0.029772967100143433, -0.014772942289710045,..."
4,N38324,health,medical,"[0.004980155732482672, 0.0042274873703718185, ..."


In [67]:
# Let's say you want to get the embeddings for the user 'U10045'
user_id_to_find = 'U10045'

# Filter the DataFrame to get the row corresponding to the specified user
user_row = users.loc[users['User'] == user_id_to_find]

# Check if the user is found in the DataFrame
if not user_row.empty:
    # Get the content (embeddings) for the specified user
    embeddings_list = user_row['Interactions'].iloc[0]
    print(embeddings_list)
else:
    print(f"User '{user_id_to_find}' not found.")


[-0.021019432693719864, -0.020645752549171448, 0.0058754319325089455, -0.0018717304337769747, -0.01582796685397625, 0.0023855387698858976, 0.0009975889697670937, -0.006993131712079048, -0.03373785689473152, -0.005475061479955912, 0.02479625679552555, 0.036673903465270996, -0.007273390889167786, 0.013665967620909214, 0.005328258965164423, 0.03066835179924965, 0.019578099250793457, 0.010416296310722828, 0.0036200133617967367, -0.0565856471657753, -0.035552866756916046, -0.014506744220852852, -0.019858358427882195, -0.01598811335861683, -0.014333250932395458, -0.015507670119404793, 0.01419979427009821, -0.010583117604255676, 0.005565145052969456, -0.0004495822940953076, -0.0023721931502223015, -0.0074068475514650345, 0.0045408643782138824, -0.02727855183184147, -0.035472795367240906, -0.02582387439906597, 0.0016206649597734213, -0.008207587525248528, 0.03707427531480789, 0.0015364204300567508, 0.017576249316334724, -0.010102673433721066, -0.010289512574672699, -0.01013603713363409, -0.024

In [78]:
content = news['Content']
content

0     [0.0059931655414402485, -0.007832585833966732,...
1     [-0.004847756586968899, -0.00798266101628542, ...
2     [-0.027519572526216507, -0.01374641340225935, ...
3     [-0.029772967100143433, -0.014772942289710045,...
4     [0.004980155732482672, 0.0042274873703718185, ...
                            ...                        
72    [-0.00848905835300684, -0.014187915250658989, ...
73    [-0.006233479827642441, 0.006515395827591419, ...
74    [0.0193341001868248, 0.007000979036092758, 0.0...
75    [0.017240002751350403, 0.0051793004386126995, ...
76    [-0.018067767843604088, 0.00041362002957612276...
Name: Content, Length: 77, dtype: object